In [ ]:
import pandas as pd
pd.set_option("display.max_columns", None)
from pprint import pprint
import warnings
warnings.filterwarnings("ignore")

### TBR六月份販促計算實戰
條件：
- FMC通路另享有TBR販促規格每條100元未稅直走價折扣
- 100條以上價格為隱藏價格，供業務彈性使用
- LSR2保持上月價格，但口數不限，賣完為止，沒有下個月補貨的規則
- TBR取消口數限制，讓客戶靈活搭配下單

### 思路拆解
> 1. 需要先將輪胎行通路和活動期間篩選出來
> 2. 將所有TBR可以享有100元未稅直走價折扣的店家篩選並建立新欄位顯示可以扣的金額
> 3. 使用groupby來用買方分組來看TBR總共買幾條
> 4. 使用pd.cut來分箱才知道客戶的條數使用那個級距的價格，不在販促的商品就是用原本的價格
> 5. LSR2使用一樣的幾個，數量不限

In [ ]:
# 載入實績
df = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\2022全年銷售\Rolling_sc_實績.xlsx")

In [ ]:
# 載入6月直走價表
tbr_price = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\價格相關\2023年\2023價格調整都在這\Июнь\202306販促直走價表.xlsx", sheet_name="TBR")
lsr_price = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\價格相關\2023年\2023價格調整都在這\Июнь\202306販促直走價表.xlsx", sheet_name="LSR2")

In [ ]:
# 建立TBR和LSR2的字典
tbr_price_dict = dict(zip(tbr_price["PCODE"], tbr_price.apply(lambda x: {"a": x["1-19"], "b": x["20-49"], "c": x["50-"], "d": x["100-"]}, axis=1)))
lsr_price_dict = dict(zip(lsr_price["PCODE"], lsr_price["Price"]))

In [ ]:
# 將販社實績拆出來
# 通路不等於B1和G（CD和販社車隊)

bssc = df[(df["實際出貨日"].between("2023-06-01", "2023-06-30")) & (~df["通路"].isin(["B1", "G"])) & (df["訂單類型"].isin(["ZSO1", "ZKE"]))]
bssc.columns = bssc.columns.str.strip()

In [ ]:
# 建立輪胎類別欄位
bssc["輪胎類別"] = bssc["中計"].map(lambda x: x[:3])

In [ ]:
# 計算FMC可以多100元
bssc["FMC單條獎勵"] = bssc.apply(lambda x: 100 if x["大胎"] in [2, 3] and x["輪胎類別"] == "TBR" else 0, axis=1)

In [ ]:
# 將bssc拆成TBR和LSR分開計算，最後再合併
tbr_bssc = bssc[bssc["輪胎類別"] == "TBR"]
lsr_bssc = bssc[bssc["輪胎類別"] == "LSR"]

In [ ]:
tbr_price_dict.keys()

In [ ]:
tbr_bssc["販促商品"] = tbr_bssc["物料"].map(lambda x: "Y" if x in tbr_price_dict.keys() else "N")

In [ ]:
tbr_bssc.shape

In [ ]:
tbr_prom = tbr_bssc[tbr_bssc["販促商品"] == "Y"]

In [ ]:
tbr_prom["販促商品總數量"] = tbr_prom.groupby("買方")["銷售數量"].transform("sum")

In [ ]:
tbr_prom.shape

In [ ]:
tbr_bssc = pd.merge(tbr_bssc, tbr_prom, how="outer")

In [ ]:
tbr_bssc["販促商品總數量"].fillna(0, inplace=True)

In [ ]:
tbr_bssc["販促商品總數量"] = tbr_bssc["販促商品總數量"].astype(int)

In [ ]:
tbr_bssc["販促商品總數量"]

In [ ]:
tbr_bssc["價格級距"] = pd.cut(tbr_bssc["販促商品總數量"], bins=[1, 20, 50, 100, 1000], right=False, labels=["a", "b", "c", "d"])

In [ ]:
tbr_price_dict

In [ ]:
def prom_price_cal(df):
    df["物料"]